In [2]:
import pandas as pd

In [3]:
def process_inactive_rows(df):
    '''
        폐업 및 삭제된 시설에 관한 데이터 제거
    '''
    _df = df.copy()
    if 'FCLTY_STATE_VALUE' in _df.columns:
        _df = _df[_df['FCLTY_STATE_VALUE'] != '폐업']
    if 'DEL_AT' in _df.columns:
        _df = _df[_df['DEL_AT'] != 'Y']
    return _df

def process_unnecessary_cols(df):
    '''
     불필요한 컬럼을 제거함
    '''
    _df = df.copy()
    drop_cols = [
        'FCLTY_MANAGE_LI_CD',
        'FCLTY_MANAGE_LI_NM',
        'ERDSGN_AT',
        'ATNM_CHCK_TRGET_AT',
        'DATA_ORIGIN_FLAG_CD',
        'DEL_AT',
    ]
    drop_cols = [i for i in drop_cols if i in _df.columns]
    _df.drop(columns=drop_cols, inplace=True)
    return _df


def process_duplicate_rows(df):
    '''
        시설명, 소재지, 시설유형명이 같은 데이터는 동일 시설로 간주하고 중복 제거
    '''
    _df = df.copy()
    def first_non_null(series):
        return series.dropna().iloc[0] if not series.dropna().empty else None
    _df = _df.groupby(
        ["FCLTY_NM", "POSESN_MBY_CTPRVN_CD", "INDUTY_NM"], as_index=False
    ).agg(first_non_null)
    return _df


In [4]:
public_facility_status = pd.read_csv('./data/raw/전국공공체육시설 데이터.csv',
                                     low_memory=False)
public_facility_status = process_inactive_rows(public_facility_status)
public_facility_status = process_unnecessary_cols(public_facility_status)
public_facility_status = process_duplicate_rows(public_facility_status)
public_facility_status.to_csv('./data/raw/전국공공체육시설 데이터_cleaned.csv', index=False)
public_facility_status.shape


(32112, 32)

In [5]:
all_facility_status = pd.read_csv('./data/raw/전국체육시설현황 데이터.csv',
                              on_bad_lines='skip',
                              low_memory=False)
# '폐업' 상태인 행 드롭
all_facility_status = process_inactive_rows(all_facility_status)
all_facility_status = process_unnecessary_cols(all_facility_status)
all_facility_status = process_duplicate_rows(all_facility_status)
all_facility_status.to_csv('./data/raw/전국체육시설현황 데이터_cleaned.csv', index=False)
all_facility_status.shape

(52716, 52)

In [6]:
all_facility_status['FCLTY_NM'].nunique()

49624

In [7]:
# 최종적으로 두 데이터프레임을 병합하고 결과값을 csv로 저장

merged_df = pd.merge(
    all_facility_status,
    public_facility_status,
    on=['FCLTY_NM', 'POSESN_MBY_CTPRVN_CD', 'INDUTY_NM'],
    how='outer',
    suffixes=('', '_public')
)

# 중복 컬럼 처리 - all_facility_status의 데이터 우선 사용
for col in merged_df.columns:
    if col.endswith('_public'):
        original_col = col[:-7]  # '_public' 제거
        # null 값인 경우에만 public_facility의 값으로 대체
        merged_df[original_col] = merged_df[original_col].fillna(merged_df[col])
        merged_df.drop(columns=[col], inplace=True)

merged_df.to_csv('./data/merged_facility_status.csv', index=False)
merged_df.shape


(53570, 63)

통합된 기존의 시설정보와 바우처 시설을 동일한 컬럼에 담는다.

In [6]:
merged_df = pd.read_csv('./data/merged_facility_status.csv', low_memory=False)

In [12]:
voucher_fac_df = pd.read_csv('./data/voucher_facilities.csv')
voucher_fac_df['voucher'] = True
voucher_fac_df.to_csv('./data/voucher_facilities.csv', index=False)

voucher_class_df = pd.read_csv('./data/voucher_classes.csv')
voucher_class_df["voucher"] = True
voucher_class_df.to_csv('./data/voucher_classes.csv', index=False)

In [ ]:
for col in voucher_fac_df.columns:
    if col not in merged_df.columns:
        print(col)

main_event_cd
city_nm
main_event_nm
city_cd
pres_nm
faci_daddr
row_num
facil_nm
facil_sn
road_addr
local_nm
faci_zip
local_cd
brno
facil_gbn_nm
res_telno
voucher


In [13]:
# 기존 퍼실리티 데이터에 맞게 컬럼명을 변경
voucher_fac_df["FCLTY_NM"] = voucher_fac_df["facil_nm"]
voucher_fac_df["FCLTY_TY_NM"] = voucher_fac_df["main_event_nm"]
voucher_fac_df["CTPRVN_NM"] = voucher_fac_df["city_nm"]
voucher_fac_df["CTPRVN_CD"] = voucher_fac_df["city_cd"]
voucher_fac_df["FULL_ADDR"] = voucher_fac_df["road_addr"] + voucher_fac_df["faci_daddr"]
voucher_fac_df["RSPNSBLTY_TEL_NO"] = voucher_fac_df["res_telno"]

In [98]:
voucher_fac_df['RSPNSBLTY_TEL_NO'].isna().sum()

np.int64(15304)

In [ ]:
# 변경한 컬럼과 불필요한 컬럼을 드롭해서, 병합을 준비
drop_cols = [
    "facil_nm",
    "main_event_nm",
    "city_nm",
    "city_cd",
    "road_addr",
    "faci_daddr",
    "res_telno",
    "local_nm",
    "faci_zip",
    "local_cd",
    "facil_gbn_nm",
    "row_num",
    "main_event_cd",

]

voucher_fac_df.drop(columns=drop_cols, inplace=True)

In [16]:
voucher_fac_df.to_csv('./data/voucher_facilities_cols_processed.csv', index=False)

In [20]:
voucher_fac_df['CTPRVN_NM'].unique()

array(['경남', '경기', '대구', '충남', '전남', '광주', '서울', '인천', '부산', '제주', '울산',
       '전북', '경북', '세종', '충북', '대전', '강원'], dtype=object)

In [24]:
def process_region_name(region_name):
    REGION_MAPPING = {
        # 특별시/광역시
        "서울특별시": "서울",
        "부산광역시": "부산",
        "대구광역시": "대구",
        "인천광역시": "인천",
        "광주광역시": "광주",
        "대전광역시": "대전",
        "울산광역시": "울산",
        # 도
        "경기도": "경기",
        "강원도": "강원",
        "충청북도": "충북",
        "충청남도": "충남",
        "전라북도": "전북",
        "전라남도": "전남",
        "경상북도": "경북",
        "경상남도": "경남",
        # 특별자치시/도
        "세종특별자치시": "세종",
        "제주특별자치도": "제주",
        "전북특별자치도": "전북",
    }
    REGION_NUMBER_MAPPING = {
        "27": "대구",
        "28": "인천",
        "43": "충북",
        "46": "전남",
        "47": "경북",
    }
    if pd.isna(region_name):
        return None
    if region_name in REGION_MAPPING:
        return REGION_MAPPING[region_name]
    if str(region_name)[:2] in REGION_NUMBER_MAPPING:  # str() 추가 및 키를 문자열로 변경
        return REGION_NUMBER_MAPPING[str(region_name)[:2]]
    return region_name  # 매핑되지 않은 경우 원래 값 반환

# 적용
merged_df['CTPRVN_NM'] = merged_df['CTPRVN_NM'].apply(process_region_name)
merged_df['CTPRVN_NM'].unique()

array(['대구', '경북', '경남', None, '충북', '전남', '경기', '충남', '전북', '서울', '대전',
       '인천', '울산', '제주', '부산', '광주', '세종', '강원'], dtype=object)

In [76]:
def merge_facility_dataframes(merged_df, voucher_df):
    """
    두 시설 데이터프레임을 FCLTY_NM(시설명)과 CTPRVN_NM(시도명)을 기준으로 병합합니다.
    merged_df의 데이터를 우선으로 하며, 중복되지 않는 voucher_df의 데이터는 새로운 행으로 추가됩니다.

    Args:
        merged_df (pd.DataFrame): 기존 시설 데이터
        voucher_df (pd.DataFrame): 바우처 시설 데이터

    Returns:
        pd.DataFrame: 병합된 데이터프레임
    """
    # 데이터프레임 복사
    _merged_df = merged_df.copy()
    _voucher_df = voucher_df.copy()

    # 병합 수행
    final_df = pd.merge(
        _merged_df,
        _voucher_df,
        on=["FCLTY_NM", "CTPRVN_NM"],
        how="outer",
        suffixes=("", "_voucher"),
    )

    # voucher 여부 표시
    final_df["is_voucher"] = final_df["voucher"].fillna(False)

    # _voucher로 끝나는 중복 컬럼 처리
    for col in final_df.columns:
        if col.endswith("_voucher"):
            original_col = col[:-8]  # '_voucher' 제거
            if original_col in final_df.columns:
                # merged_df 값이 없는 경우에만 voucher_df의 값으로 대체
                final_df[original_col] = final_df[original_col].fillna(final_df[col])
                final_df.drop(columns=[col], inplace=True)

    # voucher 컬럼 제거 (is_voucher로 대체되었으므로)
    if "voucher" in final_df.columns:
        final_df.drop(columns=["voucher"], inplace=True)

    return final_df


# 데이터프레임 병합 실행
final_facility_df = merge_facility_dataframes(merged_df, voucher_fac_df)

# 결과 저장
final_facility_df.to_csv("./data/final_facility_status.csv", index=False)

# 병합 결과 확인을 위한 기본 통계
print(f"원본 merged_df 행 수: {len(merged_df)}")
print(f"원본 voucher_df 행 수: {len(voucher_fac_df)}")
print(f"최종 병합 결과 행 수: {len(final_facility_df)}")
print(f"바우처 시설 수: {final_facility_df['is_voucher'].sum()}")


/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_72503/715073611.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df["is_voucher"] = final_df["voucher"].fillna(False)


원본 merged_df 행 수: 53570
원본 voucher_df 행 수: 22338
최종 병합 결과 행 수: 73127
바우처 시설 수: 22363


In [77]:
final_facility_df['RDNMADR_ONE_NM']=final_facility_df['RDNMADR_ONE_NM'].astype(str).replace('nan','')
final_facility_df['RDNMADR_TWO_NM']=final_facility_df['RDNMADR_TWO_NM'].astype(str).replace('nan','')
final_facility_df['FCLTY_ADDR_ONE_NM']= final_facility_df['FCLTY_ADDR_ONE_NM'].astype(str).replace('nan','')
final_facility_df['FCLTY_ADDR_TWO_NM']= final_facility_df['FCLTY_ADDR_TWO_NM'].astype(str).replace('nan','')

In [78]:
final_facility_df['FULL_ADDR'] = final_facility_df.apply(
    lambda row: row['RDNMADR_ONE_NM'] + ' ' + row['RDNMADR_TWO_NM'] if pd.isna(row['FULL_ADDR']) else row['FULL_ADDR'],
    axis=1
)
final_facility_df


,FCLTY_NM,POSESN_MBY_CTPRVN_CD,INDUTY_NM,FCLTY_SDIV_CD,FCLTY_FLAG_NM,INDUTY_CD,FCLTY_TY_CD,FCLTY_TY_NM,FCLTY_STATE_VALUE,ROAD_NM_ZIP_NO,...,ROAD_NM_EMD_NM,ROAD_NM_LI_CD,ROAD_NM_LI_NM,RDNMADR_NM,FCLTY_STATE_CD,pres_nm,facil_sn,brno,FULL_ADDR,is_voucher
0,국민재활발달센터,NaN,NaN,NaN,NaN,NaN,NaN,기타종목,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,허지원,1.0,2.229202e+09,대구광역시 수성구 범안로 543층(범물동),True
1,(구)SB복싱멀티짐,NaN,NaN,NaN,NaN,NaN,NaN,복싱,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,전성배,1.0,1.239532e+09,경기도 고양시 일산서구 가좌로 102층 SB복싱멀티짐(가좌동),True
2,Kl다이어트댄스,NaN,NaN,NaN,NaN,NaN,NaN,에어로빅,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,엄수정,1.0,6.182069e+09,경상남도 김해시 율하2로58번길 40Kl다이어트댄스(율하동),True
3,TF복싱짐,NaN,NaN,NaN,NaN,NaN,NaN,복싱,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,이준희,2.0,8.889701e+09,경상북도 경주시 유림로5번길 1493층 TF복싱짐(용강동),True
4,강성 체육관 킥복싱 무에타이,NaN,NaN,NaN,NaN,NaN,NaN,복싱,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,최재식,18888.0,3.039064e+09,충청북도 충주시 계명대로 228(연수동) 3층,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73122,힙앤업휘트니스,NaN,NaN,NaN,NaN,NaN,NaN,헬스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,호익석,20283.0,4.103129e+09,"광주광역시 광산구 장신로 72, 3층 401,402호(장덕동)",True
73123,힙필라테스,NaN,NaN,NaN,NaN,NaN,NaN,필라테스,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,김민성,1.0,2.580503e+09,인천광역시 부평구 부흥로 342덕천프라자 7층 힙필라테스(부평동),True
73124,힙핏PT,NaN,NaN,NaN,NaN,NaN,NaN,기타종목,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,강민경,1.0,5.587400e+09,경기도 양주시 회정로 120402호 힙핏(덕정동),True
73125,힛더핏 영통점,4100000000,체력단련장업,N,신고,N10,N1001,체력단련장,정상운영,16705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"경기도 수원시 영통구 반달로 31, 호원빌딩 7층 (영통동) 호원빌딩 7층 (영통동)",False


In [79]:
print(len(final_facility_df[final_facility_df['FULL_ADDR'] == ' ']))
final_facility_df['FULL_ADDR'] = final_facility_df.apply(
    lambda row: row['FCLTY_ADDR_ONE_NM'] + ' ' + row['FCLTY_ADDR_TWO_NM'] if row['FULL_ADDR'] == " " else row['FULL_ADDR'],
    axis=1
)

print(len(final_facility_df[final_facility_df['FULL_ADDR'] == ' ']))


9191
1468


Final Fac DF 에서 불필요한 컬럼들 정리한다.

In [ ]:
"""
        FCLTY_SDIV_CD : FCLTY_TY_CD에 포함됨
        INDUTY_CD : FCLTY_TY_CD에 포함됨
        FCLTY_STATE_VALUE : 모두 정상이므로 삭제함
        ROAD_NM_ZIP_NO : 우편번호 불필요 삭제
        ZIP_NO_VALUE : 우편번호 삭제
        FCLTY_ADDR_ONE_NM : 도로명주소 컬럼이 있으므로 삭제
        FCLTY_ADDR_TWO_NM : 도로명주소 컬럼이 있으므로 삭제
        CTPRVN_CD : 시도코드는 불필요 (주소가 있음)
        RDNMADR_ONE_NM : 도로명주소 컬럼이 있으므로 삭제
        RDNMADR_TWO_NM : 도로명주소 컬럼이 있으므로 삭제
        RDNMADR_THREE_NM : 도로명주소 컬럼이 있으므로 삭제
"""

In [ ]:
# Drop the specified columns from final_facility_df
columns_to_drop = [
    "FCLTY_SDIV_CD",
    "INDUTY_CD",
    "FCLTY_STATE_VALUE",
    "ROAD_NM_ZIP_NO",
    "ZIP_NO_VALUE",
    "FCLTY_ADDR_ONE_NM",
    "FCLTY_ADDR_TWO_NM",
    "CTPRVN_CD",
    "RDNMADR_ONE_NM",
    "RDNMADR_TWO_NM",
    "ADTM_CO",
    "FCLTY_AR_CO",
    "ACMD_NMPR_CO",
    "NDOR_SDIV_NM",
    "LVLH_OPN_AT",
    "LVLH_GMNSM_NM",
    "UTILIIZA_GRP_NM",
    "FCLTY_CRTN_STDR_DE",
    "ALSFC_REGIST_DE",
    "COMPET_DE",
    "SSS_DE",
    "OPER_CLSBIZ_DE",
    "FCLTY_STATE_VALUE",
    "RDNMADR_NM",
    "ROAD_NM_LI_NM",
    "RDNMADR_NM",
    "FCLTY_STATE_CD",
    "ROAD_NM_EMD_NM",
    "ROAD_NM_EMD_CD",
    "ROAD_NM_SIGNGU_NM",
    "ROAD_NM_SIGNGU_CD",
    "ROAD_NM_CTPRVN_NM",
    "ROAD_NM_CTPRVN_CD",
    "FCLTY_OPER_STLE_VALUE",
    "POSESN_MBY_CD",
    "POSESN_MBY_SIGNGU_CD",
    "ROAD_NM_LI_CD",
    "RSPNSBLTY_DEPT_NM",
    "NATION_ALSFC_AT",
    "CTPRVN_NM",
    "SIGNGU_CD",
    "SIGNGU_NM",
    "FCLTY_MANAGE_CTPRVN_CD",
    "FCLTY_MANAGE_CTPRVN_NM",
    "FCLTY_MANAGE_SIGNGU_CD",
    "FCLTY_MANAGE_SIGNGU_NM",
    "POSESN_MBY_CTPRVN_NM",
    "POSESN_MBY_SIGNGU_NM",
    "RSPNSBLTY_NM",
    "RSPNSBLTY_TEL_NO",
    "facil_sn",
    "POSESN_MBY_CTPRVN_CD",
    "FCLTY_MANAGE_EMD_CD",
    "FCLTY_MANAGE_EMD_NM",
]

filtered_drop_cols = [
    col for col in columns_to_drop if col in final_facility_df.columns
]

final_facility_df.drop(columns=filtered_drop_cols, inplace=True)
final_facility_df


,FCLTY_NM,INDUTY_NM,FCLTY_FLAG_NM,FCLTY_TY_CD,FCLTY_TY_NM,FCLTY_LO,FCLTY_LA,FCLTY_TEL_NO,FCLTY_HMPG_URL,POSESN_MBY_NM,REGIST_DT,UPDT_DT,pres_nm,brno,FULL_ADDR,is_voucher,FULL_RESPONSIBILITY_INFO
0,국민재활발달센터,NaN,NaN,NaN,기타종목,NaN,NaN,NaN,NaN,NaN,NaN,NaN,허지원,2.229202e+09,대구광역시 수성구 범안로 543층(범물동),True,NaN
1,(구)SB복싱멀티짐,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전성배,1.239532e+09,경기도 고양시 일산서구 가좌로 102층 SB복싱멀티짐(가좌동),True,NaN
2,Kl다이어트댄스,NaN,NaN,NaN,에어로빅,NaN,NaN,NaN,NaN,NaN,NaN,NaN,엄수정,6.182069e+09,경상남도 김해시 율하2로58번길 40Kl다이어트댄스(율하동),True,NaN
3,TF복싱짐,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이준희,8.889701e+09,경상북도 경주시 유림로5번길 1493층 TF복싱짐(용강동),True,NaN
4,강성 체육관 킥복싱 무에타이,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,최재식,3.039064e+09,충청북도 충주시 계명대로 228(연수동) 3층,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73122,힙앤업휘트니스,NaN,NaN,NaN,헬스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,호익석,4.103129e+09,"광주광역시 광산구 장신로 72, 3층 401,402호(장덕동)",True,NaN
73123,힙필라테스,NaN,NaN,NaN,필라테스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김민성,2.580503e+09,인천광역시 부평구 부흥로 342덕천프라자 7층 힙필라테스(부평동),True,NaN
73124,힙핏PT,NaN,NaN,NaN,기타종목,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강민경,5.587400e+09,경기도 양주시 회정로 120402호 힙핏(덕정동),True,NaN
73125,힛더핏 영통점,체력단련장업,신고,N1001,체력단련장,NaN,NaN,031-202-8282,NaN,NaN,1.678061e+12,1.721779e+12,NaN,NaN,"경기도 수원시 영통구 반달로 31, 호원빌딩 7층 (영통동) 호원빌딩 7층 (영통동)",False,NaN


In [111]:
final_facility_df['pres_nm'] = final_facility_df['pres_nm'].astype(str)

In [112]:
final_facility_df['FULL_RESPONSIBILITY_INFO'].fillna(
    final_facility_df['pres_nm'] + final_facility_df['FCLTY_TEL_NO']
, inplace=True)

/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_72503/1219518355.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_facility_df['FULL_RESPONSIBILITY_INFO'].fillna(


In [117]:
final_facility_df["FULL_RESPONSIBILITY_INFO"] = final_facility_df[
    "FULL_RESPONSIBILITY_INFO"
].astype(str).apply(lambda x: x.replace("nan", ""))
final_facility_df

,FCLTY_NM,INDUTY_NM,FCLTY_FLAG_NM,FCLTY_TY_CD,FCLTY_TY_NM,FCLTY_LO,FCLTY_LA,FCLTY_TEL_NO,FCLTY_HMPG_URL,POSESN_MBY_NM,REGIST_DT,UPDT_DT,pres_nm,brno,FULL_ADDR,is_voucher,FULL_RESPONSIBILITY_INFO
0,국민재활발달센터,NaN,NaN,NaN,기타종목,NaN,NaN,NaN,NaN,NaN,NaN,NaN,허지원,2.229202e+09,대구광역시 수성구 범안로 543층(범물동),True,
1,(구)SB복싱멀티짐,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전성배,1.239532e+09,경기도 고양시 일산서구 가좌로 102층 SB복싱멀티짐(가좌동),True,
2,Kl다이어트댄스,NaN,NaN,NaN,에어로빅,NaN,NaN,NaN,NaN,NaN,NaN,NaN,엄수정,6.182069e+09,경상남도 김해시 율하2로58번길 40Kl다이어트댄스(율하동),True,
3,TF복싱짐,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이준희,8.889701e+09,경상북도 경주시 유림로5번길 1493층 TF복싱짐(용강동),True,
4,강성 체육관 킥복싱 무에타이,NaN,NaN,NaN,복싱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,최재식,3.039064e+09,충청북도 충주시 계명대로 228(연수동) 3층,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73122,힙앤업휘트니스,NaN,NaN,NaN,헬스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,호익석,4.103129e+09,"광주광역시 광산구 장신로 72, 3층 401,402호(장덕동)",True,
73123,힙필라테스,NaN,NaN,NaN,필라테스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김민성,2.580503e+09,인천광역시 부평구 부흥로 342덕천프라자 7층 힙필라테스(부평동),True,
73124,힙핏PT,NaN,NaN,NaN,기타종목,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강민경,5.587400e+09,경기도 양주시 회정로 120402호 힙핏(덕정동),True,
73125,힛더핏 영통점,체력단련장업,신고,N1001,체력단련장,NaN,NaN,031-202-8282,NaN,NaN,1.678061e+12,1.721779e+12,nan,NaN,"경기도 수원시 영통구 반달로 31, 호원빌딩 7층 (영통동) 호원빌딩 7층 (영통동)",False,031-202-8282


In [116]:
final_facility_df.to_csv('./data/final_facility_status.csv', index=False)